# ML-Brawlhalla: Trial 3 (Work in Progress)

## Import Packages
The environment module is not included in this code, so that results cannot be abused.

In [ ]:
import numpy as np
from tensorflow import keras

from paiutils import neural_network as nn
from paiutils import reinforcement as rl
from paiutils import reinforcement_agents as ra

from environment import BrawlhallaFollow2 as Brawlhalla

## Create Environment

In [ ]:
stack_size = 3
time_size = 1
env = Brawlhalla(stack_size=stack_size)
max_steps = 100000

## Create Models

In [ ]:
inputs = keras.layers.Input((time_size, 64, 64, stack_size))
x = keras.layers.Reshape((64, 64, stack_size * time_size))(inputs)
x = nn.conv2d(32, 5, 2)(x)
x = nn.conv2d(64, 3, 2)(x)
x = nn.conv2d(128, 3, 2)(x)
x = nn.conv2d(256, 3, 2)(x)
x = nn.conv2d(512, 3, 2)(x)
x = keras.layers.Flatten()(x)
outputs = keras.layers.Dense(9, activation='softmax')(x)
model = keras.models.Model(inputs=inputs, outputs=outputs)
model.compile(optimizer=keras.optimizers.Adam(.0002), loss='mse')
model.summary()

## Create the Agent

In [ ]:
def create_memory(shape, dtype):
    if shape == (None, time_size, 64, 64, stack_size):
        return rl.ETDMemory(time_size, np.zeros(shape[2:]))
    return rl.Memory()

agent = rl.PGAgent(amodel, .99, create_memory=create_memory)

## Train the Agent

In [ ]:
agent.set_playing_data(memorizing=True, verbose=True)
env.play_episodes(agent, 10, max_steps, random=True,
                  verbose=True, episode_verbose=True)

agent.set_playing_data(
    training=True, memorizing=True, batch_size=32,
    mini_batch=12800, epochs=2, repeat=1,
    entropy_coef=0, verbose=True
)
save_dir = ''
num_episodes = 10
for ndx in range(10):
    print(f'Save Loop: {ndx}')
    result = env.play_episodes(
        agent, num_episodes, max_steps,
        verbose=True, episode_verbose=True
    )
    agent.save(save_dir, note=f'PG_{ndx}_{result}')

## Test the Agent

In [ ]:
# path = 'saves/20201223_222916_234286'
# agent.load(path, load_data=False)
agent.set_playing_data(training=False, memorizing=False)
env.play_episodes(agent, 50, 10000,verbose=True,
                  episode_verbose=False)